<a href="https://colab.research.google.com/github/Kepners/ChopOnions/blob/main/ChoppingOnionS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Block 1: Install Required Libraries

!pip install --quiet openai==0.27.8 praw==7.8.1 feedparser==6.0.10 textblob==0.17.1 \
requests cachetools==5.3.1 prettytable==3.7.0 python-dotenv==0.21.0 ratelimit==2.2.1 \
nltk==3.8.1 fuzzywuzzy==0.18.0 python-Levenshtein==0.12.2


In [12]:
# Block 2: Mount Google Drive and Load Environment Variables
from google.colab import drive
import os
from dotenv import load_dotenv

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your .env file in Google Drive
dotenv_path = '/content/drive/MyDrive/Secrets/.env'

# Load the environment variables from the .env file
load_dotenv(dotenv_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

In [13]:
# Block 3: Configure Logging
import logging
import os

# Configure Logging to write to a log file and suppress console output
log_file = 'app.log'
logging.basicConfig(
    level=logging.ERROR,  # Only log ERROR and CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler(open(os.devnull, 'w'))  # Suppress console output
    ]
)


In [14]:
# Block 4: Download NLTK Data Silently
import nltk
import sys
from contextlib import contextmanager
import os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('brown')
    nltk.download('wordnet')
    nltk.download('movie_reviews')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [15]:
# Block 5: Create Utils Directory
import os

utils_dir = 'utils'
if not os.path.exists(utils_dir):
    os.makedirs(utils_dir)


In [16]:
# Block 6: Create data_processing.py

data_processing_code = """
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textblob import TextBlob

def extract_keywords(text, max_keywords=10):
    \"""
    Extracts up to `max_keywords` nouns from the input `text`.
    \"""
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
    # Get part-of-speech tags
    tagged_words = pos_tag(filtered_words)
    # Keep nouns and proper nouns
    keywords = [word for word, pos in tagged_words if pos.startswith('NN')]
    # Limit the number of keywords
    return keywords[:max_keywords]

def analyze_sentiment(visit_count, url_count):
    \"""
    Analyzes the sentiment based on visit_count and url_count.
    Returns 'Positive', 'Negative', or 'Neutral'.
    \"""
    # Simple heuristic: higher counts imply higher sentiment
    if visit_count > 1000 and url_count > 50:
        return 'Positive'
    elif visit_count < 100 and url_count < 10:
        return 'Negative'
    else:
        return 'Neutral'
"""

with open(os.path.join('utils', 'data_processing.py'), 'w') as file:
    file.write(data_processing_code)


In [17]:
# Block 7: Initialize API Clients and Configure Caches

import sys
sys.path.append('utils')  # Add the utils directory to the system path

import praw
import openai
import logging
import warnings
from utils.data_processing import extract_keywords, analyze_sentiment
from cachetools import TTLCache, cached
import feedparser
import requests
import time
import os
from fuzzywuzzy import process

# Suppress PRAW warnings about asynchronous environments
warnings.filterwarnings("ignore", category=UserWarning, module='praw')
logging.getLogger('praw').setLevel(logging.CRITICAL)

# Initialize Reddit API using PRAW
try:
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT', 'script:TrendingTopicsScript:1.0 (by u/yourusername)')
    )
except Exception as e:
    logging.error(f"Error initializing Reddit API: {e}")

# Initialize OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')

# Initialize SerpAPI API key
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

# Initialize NewsAPI.org API key
NEWSAPI_API_KEY = os.getenv('NEWSAPI_API_KEY')

# Define a list of countries with their codes, including common abbreviations
available_countries = {
    'United States': 'US',
    'US': 'US',
    'USA': 'US',
    'India': 'IN',
    'Canada': 'CA',
    'United Kingdom': 'GB',
    'UK': 'GB',
    'Great Britain': 'GB',
    'Australia': 'AU',
    'Germany': 'DE',
    'France': 'FR',
    'Brazil': 'BR',
    'Mexico': 'MX',
    'Japan': 'JP',
    'Russia': 'RU',
    'South Korea': 'KR',
    'Korea': 'KR',
    'Italy': 'IT',
    'Spain': 'ES',
    'Netherlands': 'NL',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'Austria': 'AT',
    'Belgium': 'BE',
    'New Zealand': 'NZ'
}

# Initialize caches
post_relevance_cache = TTLCache(maxsize=1000, ttl=3600)  # Cache for 1 hour
trend_reason_cache = TTLCache(maxsize=1000, ttl=86400)   # Cache for 24 hours


In [18]:
# Block 8: Define Supporting Functions

import logging
import re
import textwrap
import requests
from cachetools import TTLCache, cached
import feedparser
import openai
from fuzzywuzzy import process
import praw
import time
import os
from textblob import TextBlob  # Ensure TextBlob is installed
from ratelimit import limits, sleep_and_retry

# Initialize OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')

# Initialize Reddit API using PRAW
reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
    user_agent=os.getenv('REDDIT_USER_AGENT')
)

# Initialize NewsAPI.org API key
NEWSAPI_API_KEY = os.getenv('NEWSAPI_API_KEY')

# Initialize SerpAPI API key
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

# Initialize caches
post_relevance_cache = TTLCache(maxsize=1000, ttl=3600)  # Cache for 1 hour
trend_reason_cache = TTLCache(maxsize=1000, ttl=86400)   # Cache for 24 hours

# SerpAPI rate limits: adjust based on your plan
MAX_CALLS_PER_MINUTE = 5  # Example limit
ONE_MINUTE = 60

@sleep_and_retry
@limits(calls=MAX_CALLS_PER_MINUTE, period=ONE_MINUTE)
def fetch_trending_topics_serpapi(country_code='US', limit=10):
    """
    Fetches trending topics using SerpAPI's Google Trends API with rate limiting.
    """
    try:
        params = {
            "engine": "google_trends",
            "geo": country_code,
            "api_key": SERPAPI_API_KEY
        }
        url = "https://serpapi.com/search"
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        trending_topics = []
        for trend in data.get('trending_searches', {}).get('searches', [])[:limit]:
            title = trend.get('title')
            visit_count = trend.get('formatted_visit_count', '0')
            url_count = trend.get('formatted_url_count', '0')
            # Convert visit_count and url_count to integers
            try:
                visit_count = int(re.sub(r'[^\d]', '', visit_count))
                url_count = int(re.sub(r'[^\d]', '', url_count))
            except:
                visit_count = 0
                url_count = 0
            sentiment = analyze_sentiment(visit_count, url_count)
            trending_topics.append({
                'title': title,
                'description': generate_trend_reason(title),
                'sentiment': sentiment
            })
        return trending_topics
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred while fetching SerpAPI Trends: {http_err}")
        return []
    except Exception as e:
        logging.error(f"Error fetching trending topics from SerpAPI: {e}")
        return []

def fetch_trending_topics_google_rss(geo='US'):
    """
    Fetches trending topics from Google Trends RSS feed.
    """
    try:
        rss_url = f'https://trends.google.com/trends/trendingsearches/daily/rss?geo={geo}'
        feed = feedparser.parse(rss_url)
        trending_topics = []
        for entry in feed.entries:
            title = entry.title
            # Optionally, extract visit_count and url_count if available in RSS
            # Since RSS might not provide these, we'll default to 0
            visit_count = 0
            url_count = 0
            sentiment = analyze_sentiment(visit_count, url_count)
            trending_topics.append({
                'title': title,
                'description': generate_trend_reason(title),
                'sentiment': sentiment
            })
        return trending_topics
    except Exception as e:
        logging.error(f"Error fetching trending topics from Google Trends RSS: {e}")
        return []

def fetch_trending_topics_newsapi(country='us', limit=10):
    """
    Fetches trending topics using NewsAPI.org top headlines.
    """
    try:
        url = f'https://newsapi.org/v2/top-headlines?country={country}&pageSize={limit}&apiKey={NEWSAPI_API_KEY}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        trending_topics = []
        for article in data.get('articles', [])[:limit]:
            title = article.get('title')
            description = article.get('description', '')
            # For visit_count and url_count, we'll use the number of times the URL appears
            # This is a proxy and may not be accurate
            url = article.get('url', '')
            url_count = 1  # Each URL is unique in this context
            visit_count = 0  # NewsAPI doesn't provide visit counts
            sentiment = analyze_sentiment(visit_count, url_count)
            trending_topics.append({
                'title': title,
                'description': generate_trend_reason(title),
                'sentiment': sentiment
            })
        return trending_topics
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred while fetching NewsAPI: {http_err}")
        return []
    except Exception as e:
        logging.error(f"Error fetching trending topics from NewsAPI: {e}")
        return []

@cached(trend_reason_cache)
def generate_trend_reason(topic, retries=3, backoff_factor=2):
    """
    Generates a brief reason why a topic is trending using OpenAI's GPT-3.5-turbo.
    """
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": "You are an assistant that provides brief reasons why a topic is trending. Your responses should be concise, in brackets, and two to four words long."
                    },
                    {
                        "role": "user",
                        "content": f"Why is '{topic}' trending? Provide a brief reason in brackets, two to four words long."
                    }
                ],
                max_tokens=20,
                temperature=0.5,
            )
            reason = response.choices[0].message.content.strip()
            return reason
        except openai.error.OpenAIError as e:
            logging.error(f"Attempt {attempt + 1} - Error generating trend reason: {e}")
            if attempt < retries - 1:
                wait_time = backoff_factor ** attempt
                logging.info(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                logging.error("Max retries reached. Using default reason.")
                return "[Trending Topic]"
        except Exception as e:
            logging.error(f"Unexpected error generating trend reason: {e}")
            return "[Trending Topic]"

def fetch_related_news(topic, limit=5):
    """
    Fetches related news articles using NewsAPI.org.
    Includes sentiment analysis for each article.
    """
    try:
        url = f'https://newsapi.org/v2/everything?q={requests.utils.quote(topic)}&sortBy=publishedAt&pageSize={limit}&apiKey={NEWSAPI_API_KEY}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        related_stories = []
        for article in data.get('articles', [])[:limit]:
            title = article.get('title', '')
            description = article.get('description', '')
            url = article.get('url', '')
            # For visit_count and url_count, we'll use the number of times the URL appears
            # This is a proxy and may not be accurate
            url_count = 1  # Each URL is unique in this context
            visit_count = 0  # NewsAPI doesn't provide visit counts
            sentiment = analyze_sentiment(visit_count, url_count)
            # Extract website source from URL
            website = re.findall(r'https?://(?:www\.)?([^/]+)/', url)
            website = website[0] if website else "Unknown Source"
            # Expand topic by extracting headline from URL or similar
            summary = generate_summary(description) if description else "No description available."
            related_stories.append({
                'type': 'News Article',
                'title': title,
                'summary': summary,
                'source': website,
                'sentiment': sentiment
            })
        return related_stories
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred while fetching related news: {http_err}")
        return []
    except Exception as e:
        logging.error(f"Error fetching related news: {e}")
        return []

@cached(post_relevance_cache)
def is_post_relevant(post_title, topic):
    """
    Determines if a Reddit post is relevant to the selected topic using OpenAI's GPT-3.5-turbo.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a helpful assistant that determines if a Reddit post is relevant to a given topic. "
                        "You should respond with 'Yes' or 'No'."
                    )
                },
                {
                    "role": "user",
                    "content": f"Is the following Reddit post relevant to the topic '{topic}'?\n\nPost Title: {post_title}\n\nRespond with 'Yes' or 'No'."
                }
            ],
            max_tokens=1,
            temperature=0,
        )
        answer = response.choices[0].message.content.strip().lower()
        return answer == 'yes'
    except Exception as e:
        logging.error(f"Error checking post relevance: {e}")
        return False

def extract_keywords_openai(text, max_keywords=5):
    """
    Extracts keywords from the text using OpenAI's GPT-3.5-turbo.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that extracts relevant keywords from a given text. Provide a list of up to five keywords separated by commas."
                },
                {
                    "role": "user",
                    "content": f"Extract up to {max_keywords} relevant keywords from the following text:\n\n{text}"
                }
            ],
            max_tokens=60,
            temperature=0.5,
        )
        keywords = response.choices[0].message.content.strip()
        # Split the keywords by commas and strip whitespace
        keyword_list = [kw.strip() for kw in keywords.split(',') if kw.strip()]
        return keyword_list
    except Exception as e:
        logging.error(f"Error extracting keywords with OpenAI: {e}")
        return []

def find_related_subreddits(keywords, limit=5):
    """
    Finds related subreddits based on a list of keywords using Reddit's API.
    """
    related_subreddits = set()
    for keyword in keywords:
        try:
            subreddits = reddit.subreddits.search(query=keyword, limit=limit)
            for subreddit in subreddits:
                # Filter out NSFW, private, and restricted subreddits
                if subreddit.over18 or subreddit.subreddit_type in ['private', 'restricted']:
                    continue
                # Additional filters: minimum subscriber count
                if subreddit.subscribers < 1000:
                    continue
                related_subreddits.add(subreddit.display_name)
        except Exception as e:
            logging.error(f"Error searching subreddits with keyword '{keyword}': {e}")
            continue
    return list(related_subreddits)[:limit]

def fetch_reddit_posts_from_subreddits(subreddits, topic, limit=10):
    """
    Fetches top posts from specified subreddits that are relevant to the topic.
    """
    reddit_posts = []
    for subreddit_name in subreddits:
        try:
            subreddit = reddit.subreddit(subreddit_name)
            search_results = subreddit.search(topic, limit=limit)
            for post in search_results:
                if post.over_18:
                    continue
                if is_post_relevant(post.title, topic):
                    sentiment = analyze_sentiment(0, 1)  # Assuming each relevant post has 1 URL
                    reddit_posts.append({
                        'type': 'Reddit Post',
                        'subreddit': subreddit_name,
                        'title': post.title,
                        'summary': f"Score: {post.score}",
                        'source': 'Reddit',
                        'sentiment': sentiment
                    })
        except Exception as e:
            logging.error(f"Error fetching posts from subreddit '{subreddit_name}': {e}")
            continue
    return reddit_posts

def extract_keywords_from_topic(topic, max_keywords=5):
    """
    Extracts keywords from the topic using OpenAI.
    """
    keywords = extract_keywords_openai(topic, max_keywords)
    # Optional: map synonyms using a predefined dictionary or OpenAI
    # For example, map 'football' to 'soccer' if relevant
    synonym_mapping = {
        'football': 'soccer',
        'election': 'voting',
        'tornadoes': 'tornado'
        # Add more mappings as needed
    }
    mapped_keywords = [synonym_mapping.get(kw.lower(), kw) for kw in keywords]
    return mapped_keywords

def find_top_subreddits(keywords, limit=5):
    """
    Finds top related subreddits based on extracted keywords.
    """
    subreddits = find_related_subreddits(keywords, limit)
    return subreddits

def analyze_sentiment(visit_count, url_count):
    """
    Analyzes the sentiment based on visit_count and url_count.
    Returns 'Positive', 'Negative', or 'Neutral'.
    """
    return analyze_sentiment(visit_count, url_count)

def generate_summary(content):
    """
    Generates a concise two-sentence summary of the provided content using OpenAI's GPT-3.5-turbo.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a concise summarizer. Provide a clear and brief two-sentence summary of the following content."
                },
                {
                    "role": "user",
                    "content": content
                }
            ],
            max_tokens=150,
            temperature=0.5,
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        return "No summary available."

def generate_fan_sentiment_summary(title, sentiment):
    """
    Generates a summary of fan sentiment from a Reddit post title using OpenAI's GPT-3.5-turbo.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an assistant that summarizes fan sentiments from online discussions into a brief overview."
                },
                {
                    "role": "user",
                    "content": f"The following Reddit post has a {sentiment} sentiment:\n\n'{title}'\n\nSummarize this sentiment for inclusion in a video script."
                }
            ],
            max_tokens=200,
            temperature=0.5,
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        logging.error(f"Error generating fan sentiment summary: {e}")
        return "No summary available."

def generate_script_for_topic(topic, content_summary, style="Normal Script"):
    """
    Generates a video script for the given topic and summary using OpenAI's GPT-3.5-turbo.
    """
    try:
        style_prompts = {
            "Flashy Script": "Create a flashy, high-energy script with quick cuts, bold visuals, and impactful statements.",
            "Expressive Script": "Compose an expressive script that evokes emotions, using descriptive language and a storytelling approach.",
            "Normal Script": "Write a straightforward script that clearly presents the information in an engaging manner."
        }
        style_prompt = style_prompts.get(style, style_prompts["Normal Script"])
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Using 'gpt-3.5-turbo' for efficiency
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a creative scriptwriter for short-form videos. "
                        "Produce engaging, dynamic scripts that are visually compelling and can be represented "
                        "using stock images and videos. The script should be suitable for a 60-second video, "
                        "have a captivating introduction, a clear narrative flow, and a strong conclusion or call-to-action. "
                        "Include specific details about the trend, such as key events, statistics, and other relevant information."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"{style_prompt}\n"
                        f"Using the following summary, write a detailed script suitable for a 60-second video:\n\n"
                        f"Summary: {content_summary}\n\n"
                        "Ensure the script includes vivid descriptions and is structured with a beginning, middle, and end. "
                        "Use language that resonates with the target audience, and make sure it's adaptable with widely available stock images and videos."
                    )
                }
            ],
            max_tokens=1500,  # Increased max_tokens for longer scripts
            temperature=0.7,
        )
        script_content = response.choices[0].message.content.strip()
        return script_content
    except Exception as e:
        logging.error(f"Error generating script: {e}")
        return "No script available."


In [19]:
# Block 9: Define the Main Function

import sys
import threading
import random
import textwrap
import time
from IPython.display import display, Markdown
from prettytable import PrettyTable
import logging
import prettytable

def main():
    # Function to display a changing message every 15 seconds
    def flashing_message(stop_event):
        messages = [
            "🔍 Gathering the latest trends...",
            "⏳ Processing data, please wait...",
            "✨ Almost there, thank you for your patience!"
        ]
        idx = 0
        while not stop_event.is_set():
            message = messages[idx % len(messages)]
            print(f"\r{message}   ", end='', flush=True)
            for _ in range(15):
                if stop_event.is_set():
                    break
                time.sleep(1)
            idx += 1
            print('\r' + ' ' * len(message) + '   ', end='', flush=True)

    # Step 1: User selects the country for trending topics
    while True:
        print("Enter the country for trending topics data (e.g., United States):")
        country_input = input("Country: ").strip()
        matching_country = get_matching_country(country_input, available_countries)
        if not matching_country:
            print("No matching countries found. Please try again.")
            # Optionally, display available countries
            print("Available countries are:")
            for country in available_countries.keys():
                print(f"- {country}")
        else:
            break

    selected_country = matching_country
    selected_country_code = available_countries[selected_country]
    print(f"You selected: {selected_country}")

    # Step 2: User selects the time frame for trending topics
    print("\nSelect the time frame for trending topics:")
    print("\033[1mA.\033[0m Last 4 hours")
    print("\033[1mB.\033[0m Last 24 hours")
    time_range_selection = input("Enter the letter of the time frame you're interested in: ").strip().upper()
    time_range_mapping = {'A': '4h', 'B': '24h'}
    time_range = time_range_mapping.get(time_range_selection)
    if not time_range:
        print("Invalid selection. Exiting.")
        return

    # Start flashing message while fetching trending topics
    stop_event = threading.Event()
    thread = threading.Thread(target=flashing_message, args=(stop_event,))
    thread.start()

    # Fetch trending topics based on selected country and time frame
    trending_topics_google = []
    trending_topics_serpapi = []
    trending_topics_newsapi = []

    if time_range == '4h':
        # For the last 4 hours, use SerpAPI and NewsAPI
        trending_topics_serpapi = fetch_trending_topics_serpapi(country_code=selected_country_code, limit=10)
        trending_topics_newsapi = fetch_trending_topics_newsapi(country=selected_country_code.lower(), limit=10)
    else:
        # For the last 24 hours, use Google Trends RSS feed
        trending_topics_google = fetch_trending_topics_google_rss(geo=selected_country_code)

    # Stop flashing message
    stop_event.set()
    thread.join()
    print()  # Move to the next line after flashing message

    # Combine trending topics from all sources
    combined_trending_topics = trending_topics_google + trending_topics_serpapi + trending_topics_newsapi

    if not combined_trending_topics:
        logging.error("No trending topics found from any source.")
        print("No trending topics found.")
        return

    # Remove duplicates based on title
    seen_titles = set()
    unique_trending_topics = []
    for topic in combined_trending_topics:
        title = topic['title']
        if title not in seen_titles:
            seen_titles.add(title)
            unique_trending_topics.append(topic)

    # Display the trending topics with additional information in a table
    print(f"\nCurrent Trending Topics in {selected_country}:\n")
    table = PrettyTable()
    table.field_names = ["No.", "Topic", "Reason", "Sentiment"]
    table.hrules = prettytable.ALL  # Add horizontal lines between rows
    table.max_width = 40  # Set maximum width for columns suitable for phone screens
    for idx, topic in enumerate(unique_trending_topics, start=1):
        title = textwrap.fill(topic['title'], width=40)
        reason = textwrap.fill(topic['description'], width=40)
        sentiment = topic.get('sentiment', 'Neutral')
        table.add_row([idx, title, reason, sentiment])
    print(table)

    # Allow the user to select a topic
    try:
        selected_idx = int(input("Enter the number of the topic you're interested in: "))
        if 1 <= selected_idx <= len(unique_trending_topics):
            selected_topic = unique_trending_topics[selected_idx - 1]['title']
            # Apply text wrapping to the selected topic message
            selected_topic_display = textwrap.fill(f"You selected: {selected_topic}", width=40)
            print(f"\n{selected_topic_display}")
        else:
            logging.error("Invalid selection. Exiting.")
            print("Invalid selection. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter a number. Exiting.")
        print("Invalid input. Please enter a number. Exiting.")
        return

    # Step 3: Fetch related news articles
    related_stories = fetch_related_news(selected_topic, limit=5)
    if related_stories:
        print("\nRelated News Articles & Reddit Posts:")
        combined_related = []
        for story in related_stories:
            combined_related.append({
                'type': 'News Article',
                'title': story['title'],
                'summary': story['summary'],
                'source': story['source'],
                'sentiment': story['sentiment']
            })
    else:
        print("\nNo related news articles found.")
        combined_related = []

    # Step 4: Search Reddit directly using the topic as a query
    print("\nSearching Reddit for posts related to the topic...")
    all_posts = []
    try:
        subreddit = reddit.subreddit('all')
        search_results = subreddit.search(selected_topic, limit=20)
        for post in search_results:
            if post.over_18:
                continue
            if is_post_relevant(post.title, selected_topic):
                sentiment = analyze_sentiment(0, 1)  # Assuming each relevant post has 1 URL
                # Extract subreddit name as source
                subreddit_name = post.subreddit.display_name
                all_posts.append({
                    'type': 'Reddit Post',
                    'title': post.title,
                    'summary': f"Score: {post.score}",
                    'source': subreddit_name,
                    'sentiment': sentiment
                })
    except Exception as e:
        logging.error(f"Error searching Reddit: {e}")

    # If no posts found, attempt to find related subreddits and fetch posts from them
    if not all_posts:
        print("No direct Reddit posts found. Attempting to find related subreddits...")
        # Extract keywords from the topic
        keywords = extract_keywords_from_topic(selected_topic, max_keywords=5)
        if keywords:
            # Find related subreddits based on keywords
            related_subreddits = find_top_subreddits(keywords, limit=5)
            if related_subreddits:
                print(f"Found related subreddits: {', '.join(related_subreddits)}")
                # Fetch posts from related subreddits
                reddit_posts = fetch_reddit_posts_from_subreddits(related_subreddits, selected_topic, limit=10)
                if reddit_posts:
                    all_posts.extend(reddit_posts)
                else:
                    print("No relevant posts found in related subreddits.")
            else:
                print("No related subreddits found based on the extracted keywords.")
        else:
            print("No keywords extracted from the selected topic.")

    # Combine news articles and Reddit posts
    combined_content = combined_related + all_posts

    if not combined_content:
        logging.error("No content available for script generation. Exiting.")
        print("\nNo content available for script generation.")
        return

    # Format the combined content
    print("\nRelated News Articles & Reddit Posts:")
    formatted_content = []
    for content in combined_content:
        title = textwrap.fill(content['title'], width=60)
        summary = textwrap.fill(content['summary'], width=60)
        source = content['source']
        formatted_content.append({
            'type': content['type'],
            'title': title,
            'summary': summary,
            'source': source,
            'sentiment': content['sentiment']
        })

    # Display the combined content in a table
    table = PrettyTable()
    table.field_names = ["No.", "Type", "Title", "Summary", "Source", "Sentiment"]
    table.hrules = prettytable.ALL  # Add horizontal lines between rows
    table.max_width = 60  # Adjust width as needed
    for idx, content in enumerate(formatted_content, start=1):
        table.add_row([
            idx,
            content['type'],
            content['title'],
            content['summary'],
            f"[{content['source']}]",  # Adding source in brackets
            content['sentiment']
        ])
    print(table)

    # Allow user to select content for script generation
    print("\nAvailable Content for Script Generation:")
    for idx, content in enumerate(formatted_content, start=1):
        print(f"{idx}. [{content['type']}] {content['title']} ({content['source']}) - Sentiment: {content['sentiment']}")

    # Shorten the input prompt and ensure it wraps properly
    input_prompt = "Enter the numbers of the items you want to generate scripts for (e.g., 1,3,5):\n"
    try:
        selected_content_input = input(input_prompt)
        selected_indices = [int(i.strip()) - 1 for i in selected_content_input.split(',') if i.strip().isdigit()]
        selected_items = [formatted_content[i] for i in selected_indices if 0 <= i < len(formatted_content)]
        if not selected_items:
            logging.error("No valid items selected. Exiting.")
            print("No valid items selected. Exiting.")
            return
    except ValueError:
        logging.error("Invalid input. Please enter numbers separated by commas. Exiting.")
        print("Invalid input. Please enter numbers separated by commas. Exiting.")
        return

    # Fetch content and generate summaries for selected items
    print("\nFetching content and generating summaries for selected items...")
    for item in selected_items:
        title = item['title']
        sentiment = item['sentiment']
        if item['type'] == 'Reddit Post':
            item['summary'] = generate_fan_sentiment_summary(title, sentiment)
        else:
            if not item['summary']:
                item['summary'] = generate_summary(item['summary'])

    # Remove stock media availability check as per your instructions
    print("\nGenerating scripts for the selected items...")
    for item in selected_items:
        title = item['title']
        summary = item.get('summary', "No summary available.")
        source = item['source']
        sentiment = item['sentiment']
        # Generate script without checking stock media
        script = generate_script_for_topic(title, summary, style="Normal Script")
        # Display the script with enhanced formatting
        script_wrapped = textwrap.fill(script, width=80)
        display(Markdown(f"### Generated Script for '{title}':\n\n{script_wrapped}\n\n**Source:** [{source}]"))
        print("====\n")

    print("\nScript execution completed.")


In [20]:
# Block 10: Run the Main Function
if __name__ == "__main__":
    main()


Enter the country for trending topics data (e.g., United States):
Country: us
You selected: US

Select the time frame for trending topics:
A. Last 4 hours
B. Last 24 hours
Enter the letter of the time frame you're interested in: b
                                   

Current Trending Topics in US:

+-----+----------------------+-----------------------------------------+-----------+
| No. |        Topic         |                  Reason                 | Sentiment |
+-----+----------------------+-----------------------------------------+-----------+
|  1  |  Election Day 2024   |       [Future political interest]       |  Neutral  |
+-----+----------------------+-----------------------------------------+-----------+
|  2  |     Quincy Jones     |         [Music legend birthday]         |  Neutral  |
+-----+----------------------+-----------------------------------------+-----------+
|  3  | When is Election Day |      [Upcoming election, confusion]     |  Neutral  |
+-----+-------------

**1. Fed eyes another rate cut**
No one knows how Tuesday's presidential
election will turn out, but the Federal
Reserve's move two days later is much
easier to predict: With inflation
continuing to cool, the Fed is set to
cut interest rates for a second time
this year.


**2. From Lime to Uber to Hertz: Free and
discounted Election Day rides**
Election Day is nearly here, and if you
haven’t already mailed in your ballot or
gone in for early voting, you might need
a ride come November 5. Here’s a roundup
of all the freebies, discounts, and
information on getting to the polls that
transportation comp…


**3. Trump Claims Pennsylvania Vote Fraud As
Harris Stumps In Michigan**
Donald Trump doubled down Sunday on his
baseless predictions of voter fraud in
battleground states as he and rival
Kamala Harris launched their frantic
final 48 hours of campaigning to court
the last holdouts in a bitterly
contested US election.


**4. Here’s When Polls Close on Election Day
in Battleground States**
It’s a tight swing state race to 270
electoral college votes as the
presidential election comes to a close.


**5. Le'Veon Bell issues threat to Kamala
Harris two days before presidential
election**
The campaigns for the 2024 presidential
election are among the most divisive in
recent history. Supporters of both
candidates spare no effort to take
points away from their rival,



Searching Reddit for posts related to the topic...

Available Content for Script Generation:
+-----+--------------+------------------------------------------+-----------+
| No. |     Type     |                  Title                   | Sentiment |
+-----+--------------+------------------------------------------+-----------+
|  1  | News Article |        Fed eyes another rate cut         |  Neutral  |
+-----+--------------+------------------------------------------+-----------+
|  2  | News Article |   From Lime to Uber to Hertz: Free and   |  Positive |
|     |              |      discounted Election Day rides       |           |
+-----+--------------+------------------------------------------+-----------+
|  3  | News Article | Trump Claims Pennsylvania Vote Fraud As  |  Neutral  |
|     |              |        Harris Stumps In Michigan         |           |
+-----+--------------+------------------------------------------+-----------+
|  4  | News Article | Here’s When Polls Close o

### Generated Expressive Script for 'Travis County: 2024 Day One Early Voter Turnout Is Higher Than Last 3 Elections 🗳️':

[Opening shot of a bustling cityscape with vibrant energy]  Narrator: In the
heart of Texas, a wave of excitement is sweeping through Travis County like a
wildfire.  [Clips of diverse individuals of all ages lining up to vote, their
faces filled with determination and hope]  Narrator: The 2024 election season
has ignited a passion for change, drawing record numbers of early voters to the
polls.  [Images of ballot boxes overflowing with votes, displaying signs of
civic engagement]  Narrator: Travis County's early voter turnout has shattered
all expectations, surpassing the combined numbers of the last three elections.
[Transition to a montage of enthusiastic supporters cheering and waving banners]
Narrator: From the young to the elderly, from all walks of life, Texans are
coming together to make their voices heard.  [Close-up shots of first-time
voters casting their ballots, capturing their sense of empowerment]  Narrator:
Every vote counts, every voice matters, and in Travis County, democracy is alive
and thriving.  [Final shot of a diverse group of people linking arms in unity]
Narrator: Join the movement, be part of history. Let your voice shape the
future. Together, we can make a difference.  [Closing frame with a call-to-
action: "Vote for change. Be the voice of tomorrow."]  [End with a fade-out as
inspirational music swells]  Narrator: Travis County: Where every vote counts,
and hope shines bright.

====


Script execution completed.
